In [1]:
import requests
import cv2
import io
import base64
import numpy as np
from PIL import Image
import paddlex as pdx
import matplotlib.pyplot as plt
import pylab
import matplotlib

matplotlib.use('TkAgg')

E:\anaconda\envs\rm\lib\site-packages\paddle\tensor\creation.py:130: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if data.dtype == np.object:


In [2]:
def get_color_map_list(num_classes):
    """ Returns the color map for visualizing the segmentation mask,
        which can support arbitrary number of classes.
    Args:
        num_classes: Number of classes
    Returns:
        The color map
    """
    color_map = num_classes * [0, 0, 0]
    for i in range(0, num_classes):
        j = 0
        lab = i
        while lab:
            color_map[i * 3] |= (((lab >> 0) & 1) << (7 - j))
            color_map[i * 3 + 1] |= (((lab >> 1) & 1) << (7 - j))
            color_map[i * 3 + 2] |= (((lab >> 2) & 1) << (7 - j))
            j += 1
            lab >>= 3
    color_map = [color_map[i:i + 3] for i in range(0, len(color_map), 3)]
    return color_map


def draw_bbox_mask(image, results, threshold=0.5, color_map=None):
    import numpy as np
    _SMALL_OBJECT_AREA_THRESH = 1000
    height, width = image.shape[:2]
    default_font_scale = max(np.sqrt(height * width) // 900, .5)
    linewidth = max(default_font_scale / 40, 2)

    labels = list()
    for dt in results:
        if dt['category'] not in labels:
            labels.append(dt['category'])

    if color_map is None:
        color_map = get_color_map_list(len(labels) + 2)[2:]
    else:
        color_map = np.asarray(color_map)
        if color_map.shape[0] != len(labels) or color_map.shape[1] != 3:
            raise Exception(
                "The shape for color_map is required to be {}x3, but recieved shape is {}x{}.".
                    format(len(labels), color_map.shape))
        if np.max(color_map) > 255 or np.min(color_map) < 0:
            raise ValueError(
                " The values in color_map should be within 0-255 range.")

    keep_results = []
    areas = []
    for dt in results:
        cname, bbox, score = dt['category'], dt['bbox'], dt['score']
        if score < threshold:
            continue
        keep_results.append(dt)
        areas.append(bbox[2] * bbox[3])
    areas = np.asarray(areas)
    sorted_idxs = np.argsort(-areas).tolist()
    keep_results = [keep_results[k]
                    for k in sorted_idxs] if keep_results else []

    for dt in keep_results:
        cname, bbox, score = dt['category'], dt['bbox'], dt['score']
        bbox = list(map(int, bbox))
        xmin, ymin, w, h = bbox
        xmax = xmin + w
        ymax = ymin + h

        color = tuple(map(int, color_map[labels.index(cname)]))
        # draw bbox
        image = cv2.rectangle(image, (xmin, ymin), (xmax, ymax), color,
                              linewidth)

        # draw mask
        if 'mask' in dt:
            mask = dt['mask'] * 255
            image = image.astype('float32')
            alpha = .7
            w_ratio = .4
            color_mask = np.asarray(color, dtype=int)
            for c in range(3):
                color_mask[c] = color_mask[c] * (1 - w_ratio) + w_ratio * 255
            idx = np.nonzero(mask)
            image[idx[0], idx[1], :] *= 1.0 - alpha
            image[idx[0], idx[1], :] += alpha * color_mask
            image = image.astype("uint8")
            contours = cv2.findContours(
                mask.astype("uint8"), cv2.RETR_CCOMP,
                cv2.CHAIN_APPROX_NONE)[-2]
            image = cv2.drawContours(
                image,
                contours,
                contourIdx=-1,
                color=color,
                thickness=1,
                lineType=cv2.LINE_AA)

        # draw label
        text_pos = (xmin, ymin)
        instance_area = w * h
        if (instance_area < _SMALL_OBJECT_AREA_THRESH or h < 40):
            if ymin >= height - 5:
                text_pos = (xmin, ymin)
            else:
                text_pos = (xmin, ymax)
        height_ratio = h / np.sqrt(height * width)
        font_scale = (np.clip((height_ratio - 0.02) / 0.08 + 1, 1.2,
                              2) * 0.5 * default_font_scale)
        text = "{} {:.2f}".format(cname, score)
        (tw, th), baseline = cv2.getTextSize(
            text,
            fontFace=cv2.FONT_HERSHEY_DUPLEX,
            fontScale=font_scale,
            thickness=1)
        image = cv2.rectangle(
            image,
            text_pos, (text_pos[0] + tw, text_pos[1] + th + baseline),
            color=color,
            thickness=-1)
        image = cv2.putText(
            image,
            text, (text_pos[0], text_pos[1] + th),
            fontFace=cv2.FONT_HERSHEY_DUPLEX,
            fontScale=font_scale,
            color=(255, 255, 255),
            thickness=1,
            lineType=cv2.LINE_AA)

    return image


In [3]:
def calc_iou(x1, y1, x2, y2, a1, b1, a2, b2):
    ax = max(x1, a1)  # 相交区域左上角横坐标
    ay = max(y1, b1)  # 相交区域左上角纵坐标
    bx = min(x2, a2)  # 相交区域右下角横坐标
    by = min(y2, b2)  # 相交区域右下角纵坐标

    area_N = (x2 - x1) * (y2 - y1)
    area_M = (a2 - a1) * (b2 - b1)

    w = max(0, bx - ax)
    h = max(0, by - ay)
    area_X = w * h

    return area_X / (area_N + area_M - area_X)

In [4]:
def nms(bbox_array):
    """
    做非极大抑制
    :param bbox_array: bbox数组，已经按照从大大到小排序
    :return: 完成非极大抑制后的数组，长度小于等于4
    """
    result_array = []

    while len(bbox_array) > 0 and len(result_array) <= 4:
        # 将置信度最高的放入
        best_bbox = bbox_array[0]
        result_array.append(best_bbox)
        del bbox_array[0]
        best_bbox_rect = list(map(int, best_bbox["bbox"]))
        index_to_del = []
        for index, bbox_item in enumerate(bbox_array):
            bbox_item_rect = list(map(int, bbox_item["bbox"]))
            if calc_iou(best_bbox_rect[0], best_bbox_rect[1],
                        best_bbox_rect[0] + best_bbox_rect[2], best_bbox_rect[1] + best_bbox_rect[3],
                        bbox_item_rect[0], bbox_item_rect[1],
                        bbox_item_rect[0] + bbox_item_rect[2], bbox_item_rect[1] + bbox_item_rect[3]
                        ) > 0.3:
                index_to_del.insert(0, index)
        for index in index_to_del:
            del bbox_array[index]

    return result_array


def filter_pip_line(result):
    tooth_dict = {
        "1": [],
        "2": [],
        "3": []
    }
    # 1.对1 - 3 进行分类
    for bbox in result:
        if bbox["category"] == "1" or bbox["category"] == "2" or bbox["category"] == "3":
            tooth_dict[bbox["category"]].append(bbox)
    # 2.每个分类挑选前四颗置信度最高的
    for item in tooth_dict.items():
        label = item[0]
        bbox_array = item[1]
        # 过滤置信度过小的
        bbox_array = list(filter(lambda bbox: bbox["score"] > 0.75, bbox_array))
        print(bbox_array)
        # 从大到小排列
        bbox_array = sorted(bbox_array, key=lambda bbox: -bbox.socre)

        # 3.如果前四颗中有IOU过大（上下是可能有交集）的，剔除置信度小的牙齿，再添加本分类的一颗牙齿，如循环到没有牙齿或者符合条件
        if len(bbox_array) > 4:
            tooth_dict[label] = nms(bbox_array)
        else:
            tooth_dict[label] = bbox_array

    print(tooth_dict)

In [6]:
result = [{'category_id': 0, 'category': '1', 'bbox': [227.37286376953125, 104.9380874633789, 57.633544921875, 66.81023406982422], 'score': 0.9609723091125488}, {'category_id': 0, 'category': '1', 'bbox': [163.3070831298828, 156.79696655273438, 42.739105224609375, 62.6927490234375], 'score': 0.29044634103775024}, {'category_id': 0, 'category': '1', 'bbox': [271.2808532714844, 99.82821655273438, 55.81500244140625, 74.45111083984375], 'score': 0.0660519152879715}, {'category_id': 0, 'category': '1', 'bbox': [230.7225799560547, 157.12030029296875, 40.25325012207031, 55.1611328125], 'score': 0.9918191432952881}, {'category_id': 0, 'category': '1', 'bbox': [198.81436157226562, 155.42398071289062, 38.685577392578125, 56.223236083984375], 'score': 0.9924766421318054}, {'category_id': 0, 'category': '1', 'bbox': [237.00448608398438, 152.57183837890625, 49.0738525390625, 73.99758911132812], 'score': 0.0589912086725235}, {'category_id': 0, 'category': '1', 'bbox': [257.6206359863281, 155.56301879882812, 45.58685302734375, 57.856109619140625], 'score': 0.06923067569732666}, {'category_id': 0, 'category': '1', 'bbox': [178.6678924560547, 107.03544616699219, 57.653839111328125, 65.1534423828125], 'score': 0.9638900756835938}, {'category_id': 0, 'category': '1', 'bbox': [217.6853485107422, 69.69682312011719, 70.15998840332031, 95.91868591308594], 'score': 0.056137021631002426}, {'category_id': 1, 'category': '2', 'bbox': [275.39105224609375, 105.76658630371094, 51.4874267578125, 65.73226928710938], 'score': 0.9897252321243286}, {'category_id': 1, 'category': '2', 'bbox': [127.05416870117188, 157.72756958007812, 46.537841796875, 61.70147705078125], 'score': 0.09893525391817093}, {'category_id': 1, 'category': '2', 'bbox': [264.30963134765625, 156.52789306640625, 37.3897705078125, 54.096954345703125], 'score': 0.9729956388473511}, {'category_id': 1, 'category': '2', 'bbox': [193.51419067382812, 158.40615844726562, 45.66571044921875, 56.449127197265625], 'score': 0.053028229624032974}, {'category_id': 1, 'category': '2', 'bbox': [111.39051818847656, 112.87391662597656, 47.213836669921875, 59.901947021484375], 'score': 0.1699638068675995}, {'category_id': 1, 'category': '2', 'bbox': [140.70071411132812, 109.97792053222656, 47.169464111328125, 64.1201171875], 'score': 0.9565659761428833}, {'category_id': 1, 'category': '2', 'bbox': [284.43670654296875, 154.41424560546875, 51.0792236328125, 55.978179931640625], 'score': 0.12301035970449448}, {'category_id': 1, 'category': '2', 'bbox': [237.52943420410156, 154.65516662597656, 57.50874328613281, 63.51275634765625], 'score': 0.05013752356171608}, {'category_id': 1, 'category': '2', 'bbox': [304.6826477050781, 107.22786712646484, 51.94708251953125, 63.50031280517578], 'score': 0.11377808451652527}, {'category_id': 1, 'category': '2', 'bbox': [164.17755126953125, 158.80126953125, 41.9072265625, 58.501007080078125], 'score': 0.9493013024330139}, {'category_id': 1, 'category': '2', 'bbox': [137.05812072753906, 69.60728454589844, 51.3994140625, 68.34602355957031], 'score': 0.05267168954014778}, {'category_id': 2, 'category': '3', 'bbox': [107.97850799560547, 107.5830307006836, 42.484046936035156, 70.2861099243164], 'score': 0.9329710602760315}, {'category_id': 2, 'category': '3', 'bbox': [181.4239044189453, 101.85650634765625, 62.8233642578125, 74.93353271484375], 'score': 0.06800007820129395}, {'category_id': 2, 'category': '3', 'bbox': [199.04403686523438, 149.78343200683594, 60.238525390625, 78.35205078125], 'score': 0.08938617259263992}, {'category_id': 2, 'category': '3', 'bbox': [314.2348937988281, 104.11164855957031, 44.33966064453125, 70.45083618164062], 'score': 0.9863510727882385}, {'category_id': 2, 'category': '3', 'bbox': [297.8144226074219, 153.53343200683594, 43.806884765625, 65.28851318359375], 'score': 0.9831740260124207}, {'category_id': 2, 'category': '3', 'bbox': [88.89092254638672, 105.90928649902344, 48.651649475097656, 71.53323364257812], 'score': 0.40171122550964355}, {'category_id': 2, 'category': '3', 'bbox': [231.21568298339844, 149.97201538085938, 67.20243835449219, 87.7825927734375], 'score': 0.12124720215797424}, {'category_id': 2, 'category': '3', 'bbox': [126.16983032226562, 158.3015594482422, 45.444061279296875, 59.578369140625], 'score': 0.9513438940048218}, {'category_id': 2, 'category': '3', 'bbox': [80.8311538696289, 112.56927490234375, 40.4501953125, 61.023101806640625], 'score': 0.10669068247079849}, {'category_id': 2, 'category': '3', 'bbox': [219.23904418945312, 100.09793090820312, 64.43243408203125, 76.84664916992188], 'score': 0.1205940917134285}, {'category_id': 2, 'category': '3', 'bbox': [107.36946868896484, 156.5475616455078, 54.226234436035156, 71.2451171875], 'score': 0.20298726856708527}, {'category_id': 2, 'category': '3', 'bbox': [251.99798583984375, 155.00790405273438, 50.80853271484375, 61.441162109375], 'score': 0.06896893680095673}, {'category_id': 2, 'category': '3', 'bbox': [257.0574645996094, 161.62586975097656, 58.31317138671875, 89.27285766601562], 'score': 0.08005829155445099}, {'category_id': 2, 'category': '3', 'bbox': [265.8634033203125, 153.14453125, 57.26348876953125, 67.13751220703125], 'score': 0.19172322750091553}, {'category_id': 2, 'category': '3', 'bbox': [99.82911682128906, 60.606014251708984, 48.16667175292969, 92.98910903930664], 'score': 0.07677807658910751}, {'category_id': 2, 'category': '3', 'bbox': [278.50213623046875, 94.49871826171875, 57.07769775390625, 80.13565063476562], 'score': 0.07257656008005142}, {'category_id': 2, 'category': '3', 'bbox': [96.0726318359375, 51.36470031738281, 49.91046142578125, 62.72709655761719], 'score': 0.19693483412265778}, {'category_id': 2, 'category': '3', 'bbox': [102.50276947021484, 154.05137634277344, 42.079811096191406, 48.8740234375], 'score': 0.06915216892957687}, {'category_id': 2, 'category': '3', 'bbox': [214.03421020507812, 49.02724838256836, 73.17462158203125, 92.26340103149414], 'score': 0.07253050804138184}, {'category_id': 2, 'category': '3', 'bbox': [314.70166015625, 156.54051208496094, 41.95599365234375, 55.035125732421875], 'score': 0.1239042803645134}, {'category_id': 2, 'category': '3', 'bbox': [309.9388122558594, 60.4041748046875, 37.0474853515625, 57.638458251953125], 'score': 0.06992527842521667}, {'category_id': 2, 'category': '3', 'bbox': [177.1994171142578, 50.56855392456055, 64.52825927734375, 76.78790664672852], 'score': 0.08419165015220642}, {'category_id': 3, 'category': '4', 'bbox': [224.807861328125, 103.68366241455078, 61.299346923828125, 73.67842864990234], 'score': 0.13670872151851654}, {'category_id': 3, 'category': '4', 'bbox': [198.57806396484375, 152.72129821777344, 65.51690673828125, 76.97409057617188], 'score': 0.137668177485466}, {'category_id': 3, 'category': '4', 'bbox': [230.06756591796875, 151.48365783691406, 69.5059814453125, 85.12957763671875], 'score': 0.11802580952644348}, {'category_id': 3, 'category': '4', 'bbox': [142.82958984375, 111.30937194824219, 53.690704345703125, 64.36251831054688], 'score': 0.08652162551879883}, {'category_id': 3, 'category': '4', 'bbox': [173.73751831054688, 156.4410858154297, 64.77130126953125, 76.92315673828125], 'score': 0.08751647919416428}, {'category_id': 3, 'category': '4', 'bbox': [183.70643615722656, 108.18870544433594, 48.051055908203125, 65.07492065429688], 'score': 0.1691150814294815}, {'category_id': 4, 'category': '5', 'bbox': [164.2422332763672, 158.10560607910156, 54.046295166015625, 63.0030517578125], 'score': 0.10791537165641785}, {'category_id': 4, 'category': '5', 'bbox': [73.63935852050781, 113.96682739257812, 49.15899658203125, 60.1636962890625], 'score': 0.08928238600492477}, {'category_id': 4, 'category': '5', 'bbox': [99.78504180908203, 113.037353515625, 54.08965301513672, 62.031646728515625], 'score': 0.2737298607826233}, {'category_id': 4, 'category': '5', 'bbox': [308.5415954589844, 104.22696685791016, 56.05126953125, 66.45980072021484], 'score': 0.07333636283874512}, {'category_id': 4, 'category': '5', 'bbox': [116.92369842529297, 110.6242904663086, 61.028343200683594, 63.51415252685547], 'score': 0.05226101726293564}, {'category_id': 4, 'category': '5', 'bbox': [116.04606628417969, 160.28065490722656, 57.844940185546875, 61.0003662109375], 'score': 0.5700551271438599}, {'category_id': 4, 'category': '5', 'bbox': [142.9431915283203, 111.25369262695312, 52.5703125, 62.407135009765625], 'score': 0.19462372362613678}, {'category_id': 5, 'category': '6', 'bbox': [92.332275390625, 160.93121337890625, 80.82470703125, 58.21527099609375], 'score': 0.3008289337158203}, {'category_id': 5, 'category': '6', 'bbox': [73.89413452148438, 116.54574584960938, 70.41682434082031, 56.000274658203125], 'score': 0.2938767075538635}, {'category_id': 5, 'category': '6', 'bbox': [63.35411071777344, 155.38768005371094, 76.70681762695312, 47.721649169921875], 'score': 0.38623565435409546}, {'category_id': 5, 'category': '6', 'bbox': [105.72782897949219, 115.11824798583984, 69.41439819335938, 57.95172882080078], 'score': 0.10202548652887344}, {'category_id': 5, 'category': '6', 'bbox': [34.00535202026367, 128.59291076660156, 77.4695930480957, 56.965240478515625], 'score': 0.07040739059448242}, {'category_id': 5, 'category': '6', 'bbox': [306.2532653808594, 107.91324615478516, 65.7705078125, 59.587623596191406], 'score': 0.07659744471311569}, {'category_id': 5, 'category': '6', 'bbox': [285.29180908203125, 159.30247497558594, 81.7989501953125, 55.936065673828125], 'score': 0.093543641269207}, {'category_id': 5, 'category': '6', 'bbox': [71.32566833496094, 71.79955291748047, 78.33670043945312, 54.01072692871094], 'score': 0.13507147133350372}, {'category_id': 5, 'category': '6', 'bbox': [41.384185791015625, 158.7572784423828, 76.85441589355469, 49.896942138671875], 'score': 0.1501995027065277}, {'category_id': 5, 'category': '6', 'bbox': [65.1803970336914, 124.97847747802734, 57.26922607421875, 54.283470153808594], 'score': 0.24546310305595398}, {'category_id': 5, 'category': '6', 'bbox': [353.48309326171875, 92.8095703125, 67.3985595703125, 54.93641662597656], 'score': 0.12282747775316238}, {'category_id': 5, 'category': '6', 'bbox': [313.49578857421875, 155.240234375, 82.555419921875, 53.212005615234375], 'score': 0.06600608676671982}, {'category_id': 5, 'category': '6', 'bbox': [57.92721176147461, 109.1142578125, 56.96652603149414, 58.7181396484375], 'score': 0.056479133665561676}, {'category_id': 5, 'category': '6', 'bbox': [10.767847061157227, 111.07986450195312, 51.00482749938965, 66.51248168945312], 'score': 0.07521702349185944}]

In [7]:
filter_pip_line(result)

[{'category_id': 0, 'category': '1', 'bbox': [227.37286376953125, 104.9380874633789, 57.633544921875, 66.81023406982422], 'score': 0.9609723091125488}, {'category_id': 0, 'category': '1', 'bbox': [230.7225799560547, 157.12030029296875, 40.25325012207031, 55.1611328125], 'score': 0.9918191432952881}, {'category_id': 0, 'category': '1', 'bbox': [198.81436157226562, 155.42398071289062, 38.685577392578125, 56.223236083984375], 'score': 0.9924766421318054}, {'category_id': 0, 'category': '1', 'bbox': [178.6678924560547, 107.03544616699219, 57.653839111328125, 65.1534423828125], 'score': 0.9638900756835938}]


AttributeError: 'dict' object has no attribute 'socre'

In [10]:
tooth_dict = {
    "1": [],
    "2": [],
    "3": []
}
# 1.对1 - 3 进行分类
for bbox in result:
    if bbox["category"] == "1" or bbox["category"] == "2" or bbox["category"] == "3":
        tooth_dict[bbox["category"]].append(bbox)
# 2.每个分类挑选前四颗置信度最高的
for item in tooth_dict.items():
    label = item[0]
    bbox_array = item[1]
    # 过滤置信度过小的
    bbox_array = list(filter(lambda bbox: bbox["score"] > 0.75, bbox_array))
    # 从大到小排列
    bbox_array = sorted(bbox_array, key=lambda bbox: -bbox["score"])

    # 3.如果前四颗中有IOU过大（上下是可能有交集）的，剔除置信度小的牙齿，再添加本分类的一颗牙齿，如循环到没有牙齿或者符合条件
    if len(bbox_array) > 4:
        tooth_dict[label] = nms(bbox_array)
    else:
        tooth_dict[label] = bbox_array

[{'category_id': 0, 'category': '1', 'bbox': [227.37286376953125, 104.9380874633789, 57.633544921875, 66.81023406982422], 'score': 0.9609723091125488}, {'category_id': 0, 'category': '1', 'bbox': [163.3070831298828, 156.79696655273438, 42.739105224609375, 62.6927490234375], 'score': 0.29044634103775024}, {'category_id': 0, 'category': '1', 'bbox': [271.2808532714844, 99.82821655273438, 55.81500244140625, 74.45111083984375], 'score': 0.0660519152879715}, {'category_id': 0, 'category': '1', 'bbox': [230.7225799560547, 157.12030029296875, 40.25325012207031, 55.1611328125], 'score': 0.9918191432952881}, {'category_id': 0, 'category': '1', 'bbox': [198.81436157226562, 155.42398071289062, 38.685577392578125, 56.223236083984375], 'score': 0.9924766421318054}, {'category_id': 0, 'category': '1', 'bbox': [237.00448608398438, 152.57183837890625, 49.0738525390625, 73.99758911132812], 'score': 0.0589912086725235}, {'category_id': 0, 'category': '1', 'bbox': [257.6206359863281, 155.56301879882812, 

In [22]:
for item in tooth_dict.values():
    print(len(item))

4
4
4


In [23]:
# predictor = pdx.deploy.Predictor('./inference_model/inference_model')
img = cv2.imread("./img/f.JPG")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
# result = predictor.predict(img)
new_result = []
for item in tooth_dict.values():
    new_result.extend(item)
plt.figure()
plt.imshow(draw_bbox_mask(img, new_result))

plt.show()

KeyboardInterrupt: 

In [21]:
draw_bbox_mask(img, new_result)

array([[[240, 187, 169],
        [236, 183, 165],
        [237, 185, 164],
        ...,
        [228, 166, 143],
        [217, 156, 137],
        [223, 162, 144]],

       [[246, 193, 175],
        [248, 196, 175],
        [240, 188, 167],
        ...,
        [212, 150, 129],
        [217, 154, 136],
        [217, 156, 138]],

       [[229, 177, 156],
        [236, 184, 163],
        [244, 192, 171],
        ...,
        [179, 114,  96],
        [204, 139, 121],
        [202, 139, 121]],

       ...,

       [[218, 170, 160],
        [219, 171, 161],
        [224, 176, 166],
        ...,
        [187, 131, 116],
        [190, 134, 121],
        [190, 134, 121]],

       [[208, 158, 147],
        [214, 164, 153],
        [222, 172, 163],
        ...,
        [188, 132, 117],
        [195, 137, 123],
        [199, 141, 127]],

       [[181, 132, 118],
        [194, 145, 131],
        [208, 158, 149],
        ...,
        [194, 138, 123],
        [186, 127, 113],
        [193, 134, 120]]

In [17]:
my_dict = {}
print(my_dict.get("1"))

None
